In [34]:
import requests

# API endpoint
url = "http://127.0.0.1:8000/find-image/"

# Request payload
payload = {
    "description": "A silver Tesla Cybertruck on the highway",  # Replace with your desired description
    "nimages": 5  # Number of images to analyze
}

# Request headers
headers = {
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)


In [35]:
response

<Response [200]>

In [ ]:

# Check the response
if response.status_code == 200:
    print("Selected Image Index:", response.json())
else:
    print("Error:", response.status_code, response.text)


In [3]:
import os
import requests

num_images = 5
query = "A silver Tesla Cybertruck on the highway"

In [4]:
BING_API_KEY = os.getenv("BING_API_KEY")

search_url = "https://api.bing.microsoft.com/v7.0/images/search"
headers = {"Ocp-Apim-Subscription-Key": BING_API_KEY}
params = {"q": query, "count": num_images}

response = requests.get(search_url, headers=headers, params=params)

In [5]:
response

<Response [200]>

In [6]:
results = response.json()
image_urls = [img["contentUrl"] for img in results.get("value", [])[:num_images]]
[print(url) for url in image_urls]

https://static1.hotcarsimages.com/wordpress/wp-content/uploads/2023/04/cybertruck.jpg
https://driveteslacanada.ca/wp-content/uploads/2019/12/Cybertruck-spotted.jpeg
https://i.kinja-img.com/image/upload/c_fill,h_675,pg_1,q_80,w_1200/a552146a9aa390bea912ba0b3cca99e0.jpg
https://s.yimg.com/ny/api/res/1.2/kM_ant_AFlqQAlYEszJCag--/YXBwaWQ9aGlnaGxhbmRlcjt3PTk2MDtoPTY4NA--/https://media.zenfs.com/en/business_insider_articles_888/136f90c113e2054d7fd2ac287f875d7a
https://www.carscoops.com/wp-content/uploads/2023/02/Tesla-Cybertruck.jpg


[None, None, None, None, None]

In [7]:
images = []
for url in image_urls:
    try:
        img_data = requests.get(url).content
        images.append(img_data)
    except Exception:
        continue  # Skip if the image can't be downloaded

In [8]:
import base64
def convert_to_base64(image_data: bytes) -> str:
    return base64.b64encode(image_data).decode("utf-8")



In [9]:
# Step 2: Convert images to base64
images_base64 = [convert_to_base64(img) for img in images]

In [10]:
prompt = (
    f"You are given {len(images_base64)} images represented as base64 strings, "
    f"and a description: '{request.description}'. Analyze the images and select the most relevant "
    f"one based on the description. Respond with the index (0-based) of the chosen image."
)

NameError: name 'request' is not defined

In [12]:
from openai import OpenAI
from schemas import SelectedImageIndex,FindImage

# Step 3: Prepare prompt and make API call
prompt = (
    f"You are given {len(images_base64)} images represented as base64 strings, "
    f"and a description: '{query}'. Analyze the images and select the most relevant "
    f"one based on the description. Respond with the index (0-based) of the chosen image."
)

client = OpenAI()  # Instantiate OpenAI client
response = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                *[
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/png;base64,{img_b64}"},
                    }
                    for img_b64 in images_base64
                ],
            ],
        }
    ],
    response_format=SelectedImageIndex,  # Enforce the integer response schema
)

# Parse the index from the response
chosen_index = response.index

AttributeError: 'ParsedChatCompletion[SelectedImageIndex]' object has no attribute 'index'

In [14]:
chosen_index = response.choices[0].message.parsed.index
chosen_index

1